# Chapter 3

# OpenAI API prompt examples

## Minimal prompt execution

In [1]:
from openai import OpenAI
import getpass

OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY') 

Enter your OPENAI_API_KEY ········


In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [12]:
prompt_input = """Write a coincise message to remind users to be vigilant about phishing attacks."""
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt_input}
  ],
  temperature= 0.7,
  max_tokens= 400  
)

print(response)

ChatCompletion(id='chatcmpl-9aQPGN5X7REwNUF5YyMwvgJYVVpTs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"Stay alert and always verify before clicking on any links or sharing personal information to protect yourself from phishing attacks."', role='assistant', function_call=None, tool_calls=None))], created=1718468342, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=32, total_tokens=54))


# Running prompts with LangChain

## Basic prompt with LangChain

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model_name="gpt-3.5-turbo")

In [14]:
prompt_input = """Write a coincise message to remind users to be vigilant about phishing attacks."""

response = llm.invoke(prompt_input)
print(response.content)

Stay alert to potential phishing attacks. Be cautious of emails or messages asking for personal information or login credentials. If in doubt, contact the organization directly to verify the request. Protect yourself from cyber threats.


# Few shot prompt with LangChain

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model_name="gpt-3.5-turbo")

In [24]:
prompt_input = """Classify the following numbers as Abra, Kadabra or Abra Kadabra:

3, 4, 5, 7, 8, 10, 11, 13, 35

Examples: 
6 // not divisible by 5, not divisible by 7 // None
15 // divisible by 5, not divisible by 7 // Abra
12 // not divisible by 5, not divisible by 7 // None
21 // not divisible by 5, divisible by 7 // Kadabra
70 // divisible by 5, divisible by 7 // Abra Kadabra
"""

response = llm.invoke(prompt_input)
print(response.content)

3 // not divisible by 5, not divisible by 7 // None
4 // not divisible by 5, not divisible by 7 // None
5 // divisible by 5, not divisible by 7 // Abra
7 // not divisible by 5, divisible by 7 // Kadabra
8 // not divisible by 5, not divisible by 7 // None
10 // divisible by 5, not divisible by 7 // Abra
11 // not divisible by 5, not divisible by 7 // None
13 // not divisible by 5, not divisible by 7 // None
35 // divisible by 5, divisible by 7 // Abra Kadabra


## Using Langchain's FewShotProm

In [25]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
  {
      "number": 6,
      "reasoning": "not divisible by 5 nor by 7",
      "result": "None"
  },
  {
      "number": 15,
      "reasoning": "divisible by 5 but not by 7",
      "result": "Abra"
  },
  {
      "number": 12,
      "reasoning": "not divisible by 5 nor by 7",
      "result": "None"
  },
  {
      "number": 21,
      "reasoning": "divisible by 7 but not by 5",
      "result": "Kadabra"
  },
  {
      "number": 70,
      "reasoning": "divisible by 5 and by 7",
      "result": "Abra Kadabra"
  } ]

example_prompt = PromptTemplate(input_variables=["number", "reasoning", "result"], template="{number} \\ {reasoning} \\ {result}")
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Classify the following numbers as Abra, Kadabra or Abra Kadabra: {comma_delimited_input_numbers}",
    input_variables=["comma_delimited_input_numbers"]
)

prompt_input = few_shot_prompt.format(comma_delimited_input_numbers="3, 4, 5, 7, 8, 10, 11, 13, 35.")

response = llm.invoke(prompt_input)
print(response.content)

3 \ Abra
4 \ Abra
5 \ Abra
7 \ Kadabra
8 \ Abra
10 \ Abra
11 \ Abra
13 \ Abra
35 \ Abra Kadabra


# Prompt templates

## Prompt template - implementing it with a Python function

In [17]:
def generate_text_summary_prompt(text, num_words, tone):
    return f'You are an experienced copywriter. Write a {num_words} words summary the the following text, using a {tone} tone: {text}'

In [30]:
segovia_aqueduct_text = "The Aqueduct of Segovia (Spanish: Acueducto de Segovia) is a Roman aqueduct in Segovia, Spain. It was built around the first century AD to channel water from springs in the mountains 17 kilometres (11 mi) away to the city's fountains, public baths and private houses, and was in use until 1973. Its elevated section, with its complete arcade of 167 arches, is one of the best-preserved Roman aqueduct bridges and the foremost symbol of Segovia, as evidenced by its presence on the city's coat of arms. The Old Town of Segovia and the aqueduct, were declared a UNESCO World Heritage Site in 1985. As the aqueduct lacks a legible inscription (one was apparently located in the structure's attic, or top portion[citation needed]), the date of construction cannot be definitively determined. The general date of the Aqueduct's construction was long a mystery, although it was thought to have been during the 1st century AD, during the reigns of the Emperors Domitian, Nerva, and Trajan. At the end of the 20th century, Géza Alföldy deciphered the text on the dedication plaque by studying the anchors that held the now missing bronze letters in place. He determined that Emperor Domitian (AD 81–96) ordered its construction[1] and the year 98 AD was proposed as the most likely date of completion.[2] However, in 2016 archeological evidence was published which points to a slightly later date, after 112 AD, during the government of Trajan or in the beginning of the government of emperor Hadrian, from 117 AD."

input_prompt = generate_text_summary_prompt(text=segovia_aqueduct_text, num_words=20, tone="knowledgeable and engaging")

response = llm.invoke(input_prompt)
print(response.content)

The Aqueduct of Segovia, a well-preserved Roman marvel, channeled water to the city's fountains and baths. UNESCO World Heritage site.


## Prompt template - using LangChain's ChatPromptTemplate

In [48]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("You are an experienced copywriter. Write a {num_words} words summary the the following text, using a {tone} tone: {text}")

In [49]:
prompt = prompt_template.format(text=segovia_aqueduct_text, num_words=20, tone="knowledgeable and engaging")

In [50]:
response = llm.invoke(prompt)
print(response.content)

The Aqueduct of Segovia, a Roman marvel built in the 1st century AD, channels water to Segovia's fountains and baths.
